params = {}
# Train the model with sample weights
model = lgb.train(params, train_data, valid_sets=[test_data], sample_weight=weights)

In [1]:

import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
from optuna.integration import LightGBMPruningCallback
import sqlite3

/home/pablo/anaconda3/envs/LaboIII/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')

In [3]:
# Abrir el archivo parquet y cargarlo en un DataFrame data/l_vm_completa_train_pendientes.parquet
df_pendientes = pd.read_parquet('./data/l_vm_completa_train_pendientes.parquet', engine='fastparquet')
# Reunir los DataFrames df_full y df_pendientes por PRODUCT_ID, CUSTOMER_ID y PERIODO, agregar las 
# columnas de df_pendientes a df_full
df_full = df_full.merge(df_pendientes, on=['PRODUCT_ID', 'CUSTOMER_ID', 'PERIODO'], how='left', suffixes=('', '_features'))
# Imprimir las columnas de df_full
print("Columnas de df_full después de la unión con df_pendientes:")
print(df_full.columns.tolist())

Columnas de df_full después de la unión con df_pendientes:
['PERIODO', 'ANIO', 'MES', 'MES_SIN', 'MES_COS', 'TRIMESTRE', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'CUSTOMER_ID', 'PRODUCT_ID', 'PLAN_PRECIOS_CUIDADOS', 'CUST_REQUEST_QTY', 'CUST_REQUEST_TN', 'TN', 'STOCK_FINAL', 'MEDIA_MOVIL_3M_CLI_PROD', 'MEDIA_MOVIL_6M_CLI_PROD', 'MEDIA_MOVIL_12M_CLI_PROD', 'DESVIO_MOVIL_3M_CLI_PROD', 'DESVIO_MOVIL_6M_CLI_PROD', 'DESVIO_MOVIL_12M_CLI_PROD', 'MEDIA_MOVIL_3M_PROD', 'MEDIA_MOVIL_6M_PROD', 'MEDIA_MOVIL_12M_PROD', 'DESVIO_MOVIL_3M_PROD', 'DESVIO_MOVIL_6M_PROD', 'DESVIO_MOVIL_12M_PROD', 'MEDIA_MOVIL_3M_CLI', 'MEDIA_MOVIL_6M_CLI', 'MEDIA_MOVIL_12M_CLI', 'DESVIO_MOVIL_3M_CLI', 'DESVIO_MOVIL_6M_CLI', 'DESVIO_MOVIL_12M_CLI', 'TN_LAG_01', 'TN_LAG_02', 'TN_LAG_03', 'TN_LAG_04', 'TN_LAG_05', 'TN_LAG_06', 'TN_LAG_07', 'TN_LAG_08', 'TN_LAG_09', 'TN_LAG_10', 'TN_LAG_11', 'TN_LAG_12', 'TN_LAG_13', 'TN_LAG_14', 'TN_LAG_15', 'CLASE', 'CLASE_DELTA', 'ORDINAL', 'TN_DELTA_01', 'TN_DELTA_02', '

In [4]:
# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
df_full['MES_PROBLEMATICO'] = df_full['PERIODO'].apply(lambda x: 1 if x in [201906, 201908] else 0)

In [5]:
# Optimizar tipos de datos numéricos
for col in df_full.select_dtypes(include=['int64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='integer')
for col in df_full.select_dtypes(include=['float64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='float')
# Variables categóricas
# categorical_features = ['ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS']
categorical_features = ['ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','PLAN_PRECIOS_CUIDADOS','MES_PROBLEMATICO']
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_full[col] = df_full[col].astype('category')
# Hacer que A_PREDECIR sea boolean si es 'S' vale True, si es 'N' False
df_full['A_PREDECIR'] = df_full['A_PREDECIR'].map({'S': True, 'N': False})


In [6]:
# Variables predictoras y objetivo
# filtrar que en X el periodo sea menor o igual a 201910
# En x eliminar la columna 'CLASE' y 'CLASE_DELTA'
X = df_full[df_full['PERIODO'] <= 201910].drop(columns=['CLASE', 'CLASE_DELTA']) 
# Filtrar en y que el periodo sea menor o igual a 201910
y = df_full[df_full['PERIODO'] <= 201910]['CLASE_DELTA']
# Eliminar df_full para liberar memoria
del df_full
gc.collect()

20

In [7]:
# Definir los periodos de validación 201910
#periodos_valid = [201910]
periodos_valid = [201910]

# Separar train y cinco conjuntos de validación respetando la secuencia temporal
X_train = X[X['PERIODO'] < periodos_valid[0]]
y_train = y[X['PERIODO'] < periodos_valid[0]]
X_val_list = [X[X['PERIODO'] == p] for p in periodos_valid]
y_val_list = [y[X['PERIODO'] == p] for p in periodos_valid]
del X, y
gc.collect()

0

In [8]:
X_train.shape

(15346065, 95)

## Optimización con Linear Trees

Se ha incluido el parámetro `linear_tree` en la optimización de hiperparámetros:

### ¿Qué es `linear_tree`?
- **Función**: Ajusta modelos de regresión lineal en las hojas de los árboles en lugar de usar valores constantes
- **Beneficios**: Mejor generalización y reducción de overfitting para datos con relaciones lineales
- **Impacto**: Puede mejorar significativamente la precisión en problemas de series temporales

### Para datos de ventas/demanda:
✅ **Ventajas**:
- Captura mejor las tendencias temporales lineales
- Reduce overfitting en patrones estacionales
- Mejora extrapolación para períodos futuros

⚠️ **Consideraciones**:
- Aumenta ligeramente el tiempo de entrenamiento
- Requiere más memoria
- Menos efectivo si las relaciones son altamente no lineales

In [12]:
import lightgbm as lgb
import optuna
from optuna.integration import LightGBMPruningCallback
import os

# === Usamos solo el primer período de validación ===
X_val = X_val_list[0]
y_val = y_val_list[0]

# Nota: Los datasets se crearán dentro de la función objective 
# para permitir cambios en linear_tree


### ⚠️ Nota importante sobre `linear_tree`

**Problema**: LightGBM no permite cambiar `linear_tree` después de que el Dataset ha sido construido.

**Solución**: Los datasets se crean dentro de la función `objective` para cada trial, permitiendo que `linear_tree` se configure correctamente para cada combinación de hiperparámetros.

**Impacto en rendimiento**: Crear datasets en cada trial añade un pequeño overhead, pero es necesario para la optimización de `linear_tree`.

In [ ]:

# Ensure required packages are installed


def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'mae',  # alias de l1
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 31, 1024, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 24),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.0, 1.0),
        'linear_tree': trial.suggest_categorical('linear_tree', [True, False]),  # Nuevo parámetro
        'verbose': -1,
        'feature_pre_filter': False,
        'bagging_seed': 42,
        'feature_fraction_seed': 42
    }

    # Crear datasets dentro de la función para permitir cambios en linear_tree
    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
    val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features)

    model = lgb.train(
        params,
        train_data,
        num_boost_round=2000,
        valid_sets=[val_data],
        callbacks=[
            lgb.early_stopping(stopping_rounds=100),
            lgb.log_evaluation(period=100),
            LightGBMPruningCallback(trial, 'l1')
        ]
    )

    best_score = model.best_score['valid_0']['l1']
    print(f"Trial {trial.number}: MAE = {best_score:.5f}")
    return best_score

# Optuna
storage_url = "sqlite:///./modelos/optuna.db"
study = optuna.create_study(
    direction='minimize',
    study_name="mae_delta_lgbm_regression_todos_los_productos_con_pendientes",
    storage=storage_url,
    load_if_exists=True
)
study.optimize(objective, n_trials=200, show_progress_bar=True)

# Mostrar mejores hiperparámetros
print("Mejores hiperparámetros encontrados:")
print(study.best_params)


[I 2025-06-24 21:06:45,821] Using an existing study with name 'mae_delta_lgbm_regression_todos_los_productos_con_pendientes' instead of creating a new one.
  0%|          | 0/200 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds


Best trial: 1. Best value: 0.0504122:   0%|          | 1/200 [00:08<27:36,  8.32s/it]

[I 2025-06-24 21:06:54,055] Trial 34 pruned. Trial was pruned at iteration 1.


Best trial: 1. Best value: 0.0504122:   0%|          | 1/200 [00:20<27:36,  8.32s/it]

...

Training until validation scores don't improve for 100 rounds


Best trial: 1. Best value: 0.0504122:   2%|▎         | 5/200 [01:35<1:02:02, 19.09s/it]

[I 2025-06-24 21:08:21,088] Trial 38 pruned. Trial was pruned at iteration 12.


Best trial: 1. Best value: 0.0504122:   3%|▎         | 6/200 [01:45<51:32, 15.94s/it]  

[I 2025-06-24 21:08:30,882] Trial 39 pruned. Trial was pruned at iteration 0.


In [ ]:
# Obtener los mejores hiperparámetros del estudio de la base de datos SQLite
storage_url = "sqlite:///./modelos/optuna.db"
study = optuna.load_study(
    study_name="mae_delta_lgbm_regression_todos_los_productos_con_pendientes",
    storage=storage_url
)
best_params = study.best_params
study.best_params

In [ ]:

# Entrenamiento final
best_params = study.best_params
best_params['objective'] = 'regression'
best_params['metric'] = 'mae'
best_params['verbose'] = -1

model_reg = lgb.train(
    best_params,
    train_data,
    num_boost_round=50000,
    valid_sets=[val_data],
    callbacks=[
        lgb.early_stopping(stopping_rounds=500),
        lgb.log_evaluation(period=500)
    ]
)

os.makedirs('./modelos', exist_ok=True)
model_reg.save_model('./modelos/lgbm_model_reg_todos_los_productos.txt')



In [ ]:
# Obtener la importancia de cada variable
importancia = model_reg.feature_importance(importance_type='gain')
nombres = X_train.columns

# Crear un DataFrame ordenado por importancia
df_importancia = pd.DataFrame({'feature': nombres, 'importance': importancia})
df_importancia = df_importancia.sort_values(by='importance', ascending=False)

# Mostrar las variables más importantes
print(df_importancia.head(50))


# Si quieres visualizarlo gráficamente:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.barh(df_importancia['feature'], df_importancia['importance'])
plt.gca().invert_yaxis()
plt.title('Importancia de variables LightGBM')
plt.xlabel('Importancia')
plt.show()

In [ ]:
# Entreno nuevamente el modelo con los mejores hiperparámetros y el conjunto completo de datos
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')
# Abrir el archivo parquet y cargarlo en un DataFrame data/l_vm_completa_train_pendientes.parquet
df_pendientes = pd.read_parquet('./data/l_vm_completa_train_pendientes.parquet', engine='fastparquet')
# Reunir los DataFrames df_full y df_pendientes por PRODUCT_ID, CUSTOMER_ID y PERIODO, agregar las 
# columnas de df_pendientes a df_full
df_full = df_full.merge(df_pendientes, on=['PRODUCT_ID', 'CUSTOMER_ID', 'PERIODO'], how='left', suffixes=('', '_features'))
# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
df_full['MES_PROBLEMATICO'] = df_full['PERIODO'].apply(lambda x: 1 if x in [201906, 201908] else 0)
# Optimizar tipos de datos numéricos
for col in df_full.select_dtypes(include=['int64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='integer')
for col in df_full.select_dtypes(include=['float64']).columns:
    df_full[col] = pd.to_numeric(df_full[col], downcast='float')
# Variables categóricas
# categorical_features = ['ANIO','MES','TRIMESTRE','ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','SKU_SIZE','CUSTOMER_ID','PRODUCT_ID','PLAN_PRECIOS_CUIDADOS']
categorical_features = ['ID_CAT1','ID_CAT2','ID_CAT3','ID_BRAND','PLAN_PRECIOS_CUIDADOS','MES_PROBLEMATICO','A_PREDECIR']
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_full[col] = df_full[col].astype('category')
# Variables predictoras y objetivo
# filtrar que en X el periodo sea menor o igual a 201910
# En x eliminar la columna 'CLASE' y 'CLASE_DELTA'
X = df_full[df_full['PERIODO'] <= 201910].drop(columns=['CLASE', 'CLASE_DELTA']) 
# Filtrar en y que el periodo sea menor o igual a 201910
y = df_full[df_full['PERIODO'] <= 201910]['CLASE_DELTA']
# Eliminar df_full para liberar memoria
del df_full
gc.collect()
# Separar train y cinco conjuntos de validación respetando la secuencia temporal
X_train = X
y_train = y
del X, y
gc.collect()

In [ ]:
# Obtener los mejores hiperparámetros del estudio de la base de datos SQLite
storage_url = "sqlite:///./modelos/optuna.db"
study1 = optuna.load_study(
    study_name="mae_delta_lgbm_regression_todos_los_productos_con_pendientes",
    storage=storage_url
)
best_params1 = study1.best_params
study1.best_params

In [ ]:

train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)

# Entrenamiento final
best_params = study1.best_params
best_params['objective'] = 'regression'
best_params['metric'] = 'mae'
best_params['verbose'] = -1

model_reg = lgb.train(
    best_params,
    train_data,
    num_boost_round=250
)

os.makedirs('./modelos', exist_ok=True)
model_reg.save_model('./modelos/lgbm_model_reg_todos_los_productos.txt')

In [ ]:
del df_full
gc.collect()

In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones
gc.collect()
df_pred_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')
df_pred_full = df_pred_full[df_pred_full['PERIODO'] == 201910].drop(columns=['CLASE', 'CLASE_DELTA'])

In [ ]:
df_pendientes = pd.read_parquet('./data/l_vm_completa_train_pendientes.parquet', engine='fastparquet')
df_pendientes = df_pendientes[df_pendientes['PERIODO'] == 201910]

In [ ]:

df_pred_full = df_pred_full.merge(df_pendientes, on=['PRODUCT_ID', 'CUSTOMER_ID', 'PERIODO'], how='left', suffixes=('', '_features'))


In [ ]:
# Mostrar los valores únicos de A_PREDECIR
print("Valores únicos de A_PREDECIR en df_pred_full:")
print(df_pred_full['A_PREDECIR'].unique())

In [ ]:
# Filtrar solo los que tengan la columna A_PREDECIR con valor 1
df_pred_full = df_pred_full[df_pred_full['A_PREDECIR'] == 'S']
# Hacer que A_PREDECIR sea boolean si es 'S' vale True, si es 'N' False
df_pred_full['A_PREDECIR'] = df_pred_full['A_PREDECIR'].map({'S': True, 'N': False})

# Agregar a df_pred_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
df_pred_full['MES_PROBLEMATICO'] = df_pred_full['PERIODO'].apply(lambda x: 1 if x in [201906, 201908] else 0)
# Convertir las variables categóricas a tipo 'category'
for col in categorical_features:
    df_pred_full[col] = df_pred_full[col].astype('category')

In [ ]:
# Eliminar del dataframe df_pred_full la columna 'PREDICCIONES'
if 'PREDICCIONES' in df_pred_full.columns:
    df_pred_full.drop(columns=['PREDICCIONES'], inplace=True)

In [ ]:

if not df_pred_full.empty:
	predictions = model_reg.predict(df_pred_full)
	df_pred_full['PREDICCIONES'] = predictions
else:
	print("df_pred_full está vacío, no se generaron predicciones.")


In [ ]:
# Mostar las columnas de df_pred_full
print("Columnas de df_pred_full después de las predicciones:")
print(df_pred_full.columns.tolist())

In [ ]:
# A cada reregistro de df_pred_full le agrego la columna que TN + PREDICCIONES
df_pred_full['TN_PREDICCIONES'] = df_pred_full['TN'] + 0.7 * df_pred_full['PREDICCIONES']
df_pred_full['TN_PREDICCIONES'] = np.where(
    df_pred_full['TN_PREDICCIONES'] <= df_pred_full['TN_MIN_12'] ,
    df_pred_full['TN_MIN_12'],
    np.where(
        df_pred_full['TN_PREDICCIONES'] >= df_pred_full['TN_MAX_12'],
        df_pred_full['TN_MAX_12'],
        df_pred_full['TN_PREDICCIONES']
    )
)   

In [ ]:
# Generar Dataframe que contenga por cada PRODUCT_ID la suma de las predicciones y la suma de la clase observada
df_final = df_pred_full.groupby('PRODUCT_ID').agg({'PREDICCIONES': 'sum', 'TN_PREDICCIONES': 'sum'}).reset_index()
# Los valore de TN_PREDICCIONES deben ser cero o mayores a cero, si no es así, los cambio a cero
df_final['TN_PREDICCIONES'] = df_final['TN_PREDICCIONES'].clip(lower=0)
df_final 


In [ ]:
# En df_final, solo dejar las columnas PRODUCT_ID, TN_PREDICCIONES que deben llamarse product_id y tn
df_final = df_final.rename(columns={'PRODUCT_ID': 'product_id', 'TN_PREDICCIONES': 'tn'})
# Eliminar el indice y PREDICCIONES
df_final = df_final[['product_id', 'tn']]   
df_final
# Guardar el DataFrame df_final en un archivo CSV
df_final.to_csv('./modelos/lgbm_model_reg_todos_los_productos.csv', index=False)
df_final.shape